In [3]:
import numpy as np

In [10]:
funct_features = np.random.rand(15, 116, 116, 1)
struct_features = np.random.rand(15, 256, 256, 256, 1)
labels = np.array([0]*5 + [1]*5 + [2]*5) 
from sklearn.model_selection import train_test_split
# Split data into training and testing sets
X_train_3d, X_test_3d, X_train_2d, X_test_2d, y_train, y_test = train_test_split(
    struct_features,
    funct_features,
    labels,
    test_size=0.2,
    random_state=42
)
# Discriminator labels (real images are labeled as 1)
real_labels_train = np.ones((X_train_3d.shape[0], 1))
real_labels_test = np.ones((X_test_3d.shape[0], 1))

# For simplicity, use real labels for reconstructed images
disc_targets_3d_train = real_labels_train
disc_targets_2d_train = real_labels_train

disc_targets_3d_test = real_labels_test
disc_targets_2d_test = real_labels_test

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight

def conv_block_3d(input_tensor, num_filters):
    x = layers.Conv3D(num_filters, (3, 3, 3), padding='valid')(input_tensor)
    x = layers.Activation('gelu')(x)
    x = layers.Conv3D(num_filters, (3, 3, 3), padding='valid')(x)
    x = layers.Activation('gelu')(x)
    return x

def encoder_block_3d(input_tensor, num_filters):
    x = conv_block_3d(input_tensor, num_filters)
    p = layers.MaxPooling3D((2, 2, 2))(x)
    return x, p

def decoder_block_3d(input_tensor, concat_tensor, num_filters):
    x = layers.Conv3DTranspose(num_filters, (2, 2, 2), strides=(2, 2, 2), padding='valid')(input_tensor)
    x = layers.concatenate([x, concat_tensor])
    x = conv_block_3d(x, num_filters)
    return x

def build_unet_3d(input_shape):
    inputs = Input(shape=input_shape)

    # Encoder
    s1, p1 = encoder_block_3d(inputs, 32)
    s2, p2 = encoder_block_3d(p1, 64)
    s3, p3 = encoder_block_3d(p2, 128)
    s4, p4 = encoder_block_3d(p3, 256)

    # Bridge
    b1 = conv_block_3d(p4, 512)

    # Decoder
    d1 = decoder_block_3d(b1, s4, 256)
    d2 = decoder_block_3d(d1, s3, 128)
    d3 = decoder_block_3d(d2, s2, 64)
    d4 = decoder_block_3d(d3, s1, 32)

    # Output
    outputs = layers.Conv3D(1, (1, 1, 1), activation='sigmoid')(d4)

    model = Model(inputs, outputs)
    return model

def build_encoder_2d(input_shape):
    inputs = Input(shape=input_shape)  # 116x116x1
    x = layers.Conv2D(32, (3, 3), activation='gelu', padding='valid')(inputs)  # 114x114x32
    x = layers.MaxPooling2D((2, 2))(x)  # 57x57x32
    x = layers.Conv2D(64, (3, 3), activation='gelu', padding='valid')(x)  # 55x55x64
    x = layers.MaxPooling2D((2, 2))(x)  # 27x27x64
    x = layers.Conv2D(128, (3, 3), activation='gelu', padding='valid')(x)  # 25x25x128
    x = layers.Flatten()(x)
    latent = layers.Dense(256, activation='gelu', kernel_regularizer=tf.keras.regularizers.l2(1e-5))(x)
    encoder = Model(inputs, latent, name='encoder_2d')
    return encoder

def build_decoder_2d(latent_dim, output_shape):
    latent_inputs = Input(shape=(latent_dim,))
    x = layers.Dense((output_shape[0] // 4) * (output_shape[1] // 4) * 64, activation='gelu')(latent_inputs)
    x = layers.Reshape((output_shape[0] // 4, output_shape[1] // 4, 64))(x)
    x = layers.Conv2DTranspose(128, (3, 3), strides=2, padding='valid', activation='gelu')(x)
    x = layers.Conv2DTranspose(64, (3, 3), strides=2, padding='valid', activation='gelu')(x)
    x = layers.Conv2DTranspose(1, (3, 3), padding='valid', activation='sigmoid')(x)
    outputs = layers.Cropping2D(cropping=((2, 3), (2, 3)))(x)
    assert outputs.shape[1:-1] == output_shape, f"Output shape {outputs.shape[1:-1]} does not match expected shape {output_shape}"
    decoder = Model(latent_inputs, outputs, name='decoder_2d')
    return decoder

def build_discriminator_3d(input_shape):
    inputs = Input(shape=input_shape)
    x = layers.Conv3D(32, (3, 3, 3), activation='gelu')(inputs)
    x = layers.MaxPooling3D((2, 2, 2))(x)
    x = layers.Conv3D(64, (3, 3, 3), activation='gelu')(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    discriminator = Model(inputs, outputs, name='discriminator_3d')
    return discriminator

def build_discriminator_2d(input_shape):
    inputs = Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='gelu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='gelu')(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    discriminator = Model(inputs, outputs, name='discriminator_2d')
    return discriminator

def build_classifier(input_dim, num_classes):
    inputs = Input(shape=(input_dim,))
    x = layers.Dense(256, activation='gelu', kernel_regularizer=tf.keras.regularizers.l2(1e-5))(inputs)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='gelu', kernel_regularizer=tf.keras.regularizers.l2(1e-5))(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    classifier = Model(inputs, outputs, name='classifier')
    return classifier

# Input shapes
input_shape_3d = (169, 205, 169, 1)
input_shape_2d = (116, 116, 1)
latent_dim = 256
num_classes = 3

# Build models
unet_3d = build_unet_3d(input_shape_3d)
encoder_2d = build_encoder_2d(input_shape_2d)
decoder_2d = build_decoder_2d(latent_dim, input_shape_2d[:-1])
discriminator_3d = build_discriminator_3d(input_shape_3d)
discriminator_2d = build_discriminator_2d(input_shape_2d)
classifier = build_classifier(latent_dim * 2, num_classes)

# Inputs
input_3d = Input(shape=input_shape_3d)
input_2d = Input(shape=input_shape_2d)

# Encoding with U-Net for 3D data
reconstructed_3d = unet_3d(input_3d)

# Global Average Pooling to get latent representation
latent_3d = layers.GlobalAveragePooling3D()(reconstructed_3d)

# Encoding for 2D data
latent_2d = encoder_2d(input_2d)

# Concatenate the two latent representations
combined_latent = layers.Concatenate()([latent_3d, latent_2d])

# Decoding for 2D data
reconstructed_2d = decoder_2d(latent_2d)

# Discriminator outputs
disc_output_3d = discriminator_3d(reconstructed_3d)
disc_output_2d = discriminator_2d(reconstructed_2d)

# Classification output
classification_output = classifier(combined_latent)

# Define the combined model
model = Model(
    inputs=[input_3d, input_2d],
    outputs=[
        reconstructed_3d,
        reconstructed_2d,
        disc_output_3d,
        disc_output_2d,
        classification_output
    ]
)

# Loss weights
lambda_reconstruction = 1.0
lambda_adversarial = 0.1
lambda_classification = 1.0

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'reconstructed_3d': 'mse',
        'reconstructed_2d': 'mse',
        'disc_output_3d': 'binary_crossentropy',
        'disc_output_2d': 'binary_crossentropy',
        'classifier': 'sparse_categorical_crossentropy'
    },
    loss_weights={
        'reconstructed_3d': lambda_reconstruction,
        'reconstructed_2d': lambda_reconstruction,
        'disc_output_3d': lambda_adversarial,
        'disc_output_2d': lambda_adversarial,
        'classifier': lambda_classification
    },
    metrics={
        'classifier': 'accuracy'
    }
)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Discriminator labels (real images are labeled as 1)
real_labels = np.ones((struct_features.shape[0], 1))
fake_labels = np.zeros((struct_features.shape[0], 1))

# For simplicity, use real labels for reconstructed images (you can adjust as needed)
disc_targets_3d_train = real_labels[:len(X_train_3d)]
disc_targets_2d_train = real_labels[:len(X_train_2d)]
disc_targets_3d_test = real_labels[len(X_train_3d):]
disc_targets_2d_test = real_labels[len(X_train_2d):]

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Compute sample weights
sample_weights = np.array([class_weights_dict[label] for label in labels])

# Train the model
history = model.fit(
    [X_train_3d, X_train_2d],
    {
        'reconstructed_3d': X_train_3d,
        'reconstructed_2d': X_train_2d,
        'disc_output_3d': disc_targets_3d_train,
        'disc_output_2d': disc_targets_2d_train,
        'classifier': y_train
    },
    sample_weight={
        'classifier': sample_weights[:len(X_train_3d)]
    },
    epochs=50,
    batch_size=8,
    validation_data=(
        [X_test_3d, X_test_2d],
        {
            'reconstructed_3d': X_test_3d,
            'reconstructed_2d': X_test_2d,
            'disc_output_3d': disc_targets_3d_test,
            'disc_output_2d': disc_targets_2d_test,
            'classifier': y_test
        }
    ),
    callbacks=[early_stopping, lr_scheduler]
)

# Evaluate the model on the test set
evaluation = model.evaluate(
    [X_test_3d, X_test_2d],
    {
        'reconstructed_3d': X_test_3d,
        'reconstructed_2d': X_test_2d,
        'disc_output_3d': disc_targets_3d_test,
        'disc_output_2d': disc_targets_2d_test,
        'classifier': y_test
    },
    verbose=0
)

classification_accuracy = evaluation[model.metrics_names.index('classifier_accuracy')]

print(f'Classifier Test Accuracy: {classification_accuracy * 100:.2f}%')

In [14]:

# Input shapes
input_shape_3d = (256, 256, 256, 1)
input_shape_2d = (116, 116, 1)
latent_dim = 256
num_classes = 3

# Build models
unet_3d = build_unet_3d(input_shape_3d)
encoder_2d = build_encoder_2d(input_shape_2d)
decoder_2d = build_decoder_2d(latent_dim, input_shape_2d[:-1])
classifier = build_classifier(latent_dim * 2, num_classes)

# Inputs
input_3d = Input(shape=input_shape_3d)
input_2d = Input(shape=input_shape_2d)

# Encoding with U-Net for 3D data
reconstructed_3d = unet_3d(input_3d)

# Global Average Pooling to get latent representation
latent_3d = layers.GlobalAveragePooling3D()(reconstructed_3d)

# Encoding for 2D data
latent_2d = encoder_2d(input_2d)

# Concatenate the two latent representations
combined_latent = layers.Concatenate()([latent_3d, latent_2d])

# Decoding for 2D data
reconstructed_2d = decoder_2d(latent_2d)

# Classification output
classification_output = classifier(combined_latent)

# Define the combined model
model = Model(
    inputs=[input_3d, input_2d],
    outputs=[
        reconstructed_3d,
        reconstructed_2d,
        classification_output
    ]
)


ValueError: Exception encountered when calling layer "classifier" (type Functional).

Input 0 of layer "dense_2" is incompatible with the layer: expected axis -1 of input shape to have value 512, but received input with shape (None, 257)

Call arguments received by layer "classifier" (type Functional):
  • inputs=tf.Tensor(shape=(None, 257), dtype=float32)
  • training=None
  • mask=None

In [ ]:

# Loss weights
lambda_reconstruction = 1.0
lambda_classification = 1.0

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'reconstructed_3d': 'mse',
        'reconstructed_2d': 'mse',
        'classifier': 'sparse_categorical_crossentropy'
    },
    loss_weights={
        'reconstructed_3d': lambda_reconstruction,
        'reconstructed_2d': lambda_reconstruction,
        'classifier': lambda_classification
    },
    metrics={
        'classifier': 'accuracy'
    }
)


In [ ]:

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Discriminator labels (real images are labeled as 1)
real_labels = np.ones((struct_features.shape[0], 1))
fake_labels = np.zeros((struct_features.shape[0], 1))

# For simplicity, use real labels for reconstructed images (you can adjust as needed)
disc_targets_3d = real_labels

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Compute sample weights
sample_weights = np.array([class_weights_dict[label] for label in labels])


In [ ]:

# Train the model
history = model.fit(
    [X_train_3d, X_train_2d],
    {
        'reconstructed_3d': X_train_3d,
        'reconstructed_2d': X_train_2d,
        'classifier': y_train
    },
    sample_weight={
        'classifier': sample_weights
    },
    epochs=50,
    batch_size=8,
    validation_data=(
        [X_test_3d, X_test_2d],
        {
            'reconstructed_3d': X_test_3d,
            'reconstructed_2d': X_test_2d,
            'classifier': y_test
        }
    ),
    callbacks=[early_stopping, lr_scheduler]
)

In [ ]:
# Evaluate the model on the test set
evaluation = model.evaluate(
    [X_test_3d, X_test_2d],
    {
        'reconstructed_3d': X_test_3d,
        'reconstructed_2d': X_test_2d,
        'classifier': y_test
    },
    verbose=0
)

classification_accuracy = evaluation[model.metrics_names.index('classifier_accuracy')]

print(f'Classifier Test Accuracy: {classification_accuracy * 100:.2f}%')